# Advent of Code 2020 Day 13
[link](https://adventofcode.com/2020/day/13)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#read-input" data-toc-modified-id="read-input-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>read input</a></span></li><li><span><a href="#part-1" data-toc-modified-id="part-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>part 1</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li><li><span><a href="#part-2" data-toc-modified-id="part-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 2</a></span><ul class="toc-item"><li><span><a href="#solution-1" data-toc-modified-id="solution-1-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>solution 1</a></span></li><li><span><a href="#solution-2" data-toc-modified-id="solution-2-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>solution 2</a></span></li><li><span><a href="#answer" data-toc-modified-id="answer-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>answer</a></span></li></ul></li></ul></div>

## read input

This problem benefits if we ignore the x but remember the original index of the non-x. This lossless cleaning resulted in a cleaner more useful data.

In [1]:
function parse_input(filename)
    lines = readlines(filename)
    earliest_depart_time = parse(Int, lines[1])
    buses_string = split(lines[2], ',')
    positions = findall(!=("x"), buses_string)
    buses = parse.(Int, buses_string[positions])
    return (; earliest_depart_time, positions, buses)
end

parse_input (generic function with 1 method)

In [2]:
input_sample_1 = parse_input("input_sample_1.txt")

(earliest_depart_time = 939, positions = [1, 2, 5, 7, 8], buses = [7, 13, 59, 31, 19])

In [3]:
input_puzzle = parse_input("input_puzzle.txt")

(earliest_depart_time = 1000511, positions = [1, 24, 30, 47, 48, 49, 53, 61, 102], buses = [29, 37, 409, 17, 13, 19, 23, 353, 41])

## part 1

In [4]:
function depart(earliest_depart_time, buses)    
    wait_times = buses .- rem.(earliest_depart_time, buses)
    i = argmin(wait_times)
    return (t = wait_times[i], bus = buses[i])
end

depart (generic function with 1 method)

### answer

In [5]:
function show_answer_report(input, ::Val{:part1})
    t, bus = depart(input.earliest_depart_time, input.buses)
    @info "Answer found." answer= t * bus
    return
end

show_answer_report (generic function with 1 method)

In [6]:
show_answer_report(input_sample_1, Val(:part1))

┌ Info: Answer found.
│   answer = 295
└ @ Main In[5]:3


In [7]:
show_answer_report(input_puzzle, Val(:part1))

┌ Info: Answer found.
│   answer = 222
└ @ Main In[5]:3


## part 2

### solution 1

This smells of Chinese Remainder Theorem. I could implement it myself (which is as below), or use `CRT` in `Mods.jl`.

In [16]:
"""
Get the value `x` such that `x ≡ a[i] mod n[i]` for each index `i` in `congruent_pairs.
"""
function chinese_remainder_theorem(a, n)
    a, n
    N = prod(n)
    y = N .÷ n
    z = invmod.(y, n)
    x = sum(a .* y .* z)

    return mod(x, N)
end

chinese_remainder_theorem

### solution 2

We deliberately implement a solution without Chinese Remainder Theorem to prove that it is possible. In the event that the size is too big, we should use `BigInt` instead of `Int`, but the Advent of Code input is very contained so it's fine.

In [19]:
function get_timestamp(time_lefts::Vector{Int}, buses::Vector{Int})
    t = 0
    N = 1

    for i in 1:length(time_lefts)
        while true
            t += N
            (mod(t - time_lefts[i], buses[i]) == 0) && break
        end
        N *= buses[i]
    end
    
    return mod(t, N)
end

get_timestamp (generic function with 1 method)

In [20]:
function get_earliest_depart_time(positions, buses)
    time_lefts = buses .- positions .+ 1
    return get_timestamp(time_lefts, buses)
    # alternatively, use chinese_remainder_theorem(time_Lefts, valid_buses) instead
end

get_earliest_depart_time (generic function with 1 method)

### answer

In [13]:
function show_answer_report(input, ::Val{:part2})
    @info "Answer found." answer=get_earliest_depart_time(input.positions, input.buses)
    return
end

show_answer_report (generic function with 2 methods)

In [21]:
show_answer_report(input_sample_1, Val(:part2))

┌ Info: Answer found.
│   answer = 1068781
└ @ Main In[13]:2


In [22]:
show_answer_report(input_puzzle, Val(:part2))

┌ Info: Answer found.
│   answer = 408270049879073
└ @ Main In[13]:2
